Imports

In [1]:
import sys
from importlib.metadata import version

import ast
import pandas as pd
import numpy as np



from helper import (
    TransformationStrainCalculator,
    DataHandler,
    FeatureGenerator,
    deformation_matrices,
    LambdaCalculator,
    Lambda2Model,
)


from catboost import CatBoostRegressor
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("Python version: " + sys.version)
print("NumPy version: {}".format(version("numpy")))
print("pandas version: {}".format(version("pandas")))
print("SymPy version: {}".format(version("sympy")))
print("scikit-learn version: {}".format(version("scikit-learn")))
print("CatBoost version: {}".format(version("catboost")))
print("CBFV version: {}".format(version("CBFV")))

c:\Users\sinah\Documents\ts\Transformation-Strain-Model-NiTi\helper.py:615: SyntaxWarning: invalid escape sequence '\d'
  .replace("\d+", "", regex=True)


Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
NumPy version: 1.26.4
pandas version: 2.2.2
SymPy version: 1.13.0
scikit-learn version: 1.5.1
CatBoost version: 1.2.5
CBFV version: 1.1.0


Calculate deformation matrices

    - B19

In [2]:
matrices = deformation_matrices()
matrices.print_B19()


B19:
B1:
⎡a⋅cos(β)  -c     c  ⎤
⎢────────  ────  ────⎥
⎢   a₀     2⋅a₀  2⋅a₀⎥
⎢                    ⎥
⎢a⋅sin(β)            ⎥
⎢────────   0     0  ⎥
⎢   a₀               ⎥
⎢                    ⎥
⎢           b     b  ⎥
⎢   0      ────  ────⎥
⎣          2⋅a₀  2⋅a₀⎦

B2:
⎡a⋅cos(β)  -c    -c  ⎤
⎢────────  ────  ────⎥
⎢   a₀     2⋅a₀  2⋅a₀⎥
⎢                    ⎥
⎢a⋅sin(β)            ⎥
⎢────────   0     0  ⎥
⎢   a₀               ⎥
⎢                    ⎥
⎢          -b     b  ⎥
⎢   0      ────  ────⎥
⎣          2⋅a₀  2⋅a₀⎦

B3:
⎡ c    a⋅cos(β)  -c  ⎤
⎢────  ────────  ────⎥
⎢2⋅a₀     a₀     2⋅a₀⎥
⎢                    ⎥
⎢      a⋅sin(β)      ⎥
⎢ 0    ────────   0  ⎥
⎢         a₀         ⎥
⎢                    ⎥
⎢ b               b  ⎥
⎢────     0      ────⎥
⎣2⋅a₀            2⋅a₀⎦

B4:
⎡-c    a⋅cos(β)  -c  ⎤
⎢────  ────────  ────⎥
⎢2⋅a₀     a₀     2⋅a₀⎥
⎢                    ⎥
⎢      a⋅sin(β)      ⎥
⎢ 0    ────────   0  ⎥
⎢         a₀         ⎥
⎢                    ⎥
⎢ b              -b  ⎥
⎢────     

    - B19'

In [3]:
matrices.print_B19p()


B19':
B1:
⎡a⋅cos(β)  -c     c  ⎤
⎢────────  ────  ────⎥
⎢   a₀     2⋅a₀  2⋅a₀⎥
⎢                    ⎥
⎢a⋅sin(β)            ⎥
⎢────────   0     0  ⎥
⎢   a₀               ⎥
⎢                    ⎥
⎢           b     b  ⎥
⎢   0      ────  ────⎥
⎣          2⋅a₀  2⋅a₀⎦

B1':
⎡-a⋅cos(β)   -c     c  ⎤
⎢──────────  ────  ────⎥
⎢    a₀      2⋅a₀  2⋅a₀⎥
⎢                      ⎥
⎢-a⋅sin(β)             ⎥
⎢──────────   0     0  ⎥
⎢    a₀                ⎥
⎢                      ⎥
⎢            -b    -b  ⎥
⎢    0       ────  ────⎥
⎣            2⋅a₀  2⋅a₀⎦

B2:
⎡a⋅cos(β)  -c    -c  ⎤
⎢────────  ────  ────⎥
⎢   a₀     2⋅a₀  2⋅a₀⎥
⎢                    ⎥
⎢a⋅sin(β)            ⎥
⎢────────   0     0  ⎥
⎢   a₀               ⎥
⎢                    ⎥
⎢          -b     b  ⎥
⎢   0      ────  ────⎥
⎣          2⋅a₀  2⋅a₀⎦

B2':
⎡-a⋅cos(β)   -c    -c  ⎤
⎢──────────  ────  ────⎥
⎢    a₀      2⋅a₀  2⋅a₀⎥
⎢                      ⎥
⎢-a⋅sin(β)             ⎥
⎢──────────   0     0  ⎥
⎢    a₀                ⎥
⎢                

Calculate theoretical transformation strain for NiTi SMAs martensite variants in a given direction

In [4]:
calculator = TransformationStrainCalculator()
a0 = 3.1431
a = 3.10992
b = 4.14056
c = 4.88988
beta = 100.493
deformation_direction = [1, 1, 0]

calculator.set_lattice_constants_and_beta(a0, a, b, c, beta)

calculator.set_custom_directions(deformation_direction)
calculator.print_results()


Deformation direction: [1 1 0]
V1: 7.19785%
V1': -6.79041%
V2: 7.19785%
V2': -6.79041%
V3: -6.79041%
V3': 7.19785%
V4: 7.19785%
V4': -6.79041%
V5: -6.84935% <- Lowest!
V5': -6.84935% <- Lowest!
V6: 10.00819% <- Highest!
V6: 10.00819%
V6': 10.00819% <- Highest!
V6': 10.00819%
Maximum transformation strain for tension in direction [1 1 0]: 10.00819% 
Maximum transformation strain for compression in direction [1 1 0]: 6.84935% 


Calculate theoretical transformation strain for NiTi SMAs for different deformation directions and deformation modes

In [5]:
data = {
    "a0 (A)": [3.1431],
    "a (A)": [3.10992],
    "b (A)": [4.14056],
    "c (A)": [4.88988],
    "beta": [100.493],
}
directions = [
    [1, 1, 1],
    [0, 1, 1],
]

data_handler = DataHandler(data)
df = data_handler.generate_dataframe()

results = []
for index, row in df.iterrows():
    # Initialize the calculator
    calculator = TransformationStrainCalculator()

    for deformation_direction in directions:
        # Setup calculator with data from the row
        calculator.set_lattice_constants_and_beta(
            row["a0 (A)"], row["a (A)"], row["b (A)"], row["c (A)"], row["beta"]
        )
        calculator.set_custom_directions(deformation_direction)

        # Calculate max strain and other information
        max_strains_info = calculator.calc_max_strain_and_info()

        # Append row for tension
        tension_row = row.copy()
        tension_row["deformation_direction"] = str(deformation_direction)
        tension_row["deformation_type"] = "tension"
        tension_row["theoretical_transformation_strain"] = max_strains_info[0][0]
        results.append(tension_row)

        # Append row for compression
        compression_row = row.copy()
        compression_row["deformation_direction"] = str(deformation_direction)
        compression_row["deformation_type"] = "compression"
        compression_row["theoretical_transformation_strain"] = max_strains_info[0][1]
        results.append(compression_row)

new_df = pd.DataFrame(results)
new_df


,a0 (A),a (A),b (A),c (A),beta,deformation_direction,deformation_type,theoretical_transformation_strain
0,3.1431,3.10992,4.14056,4.88988,100.493,"[1, 1, 1]",tension,14.891677
0,3.1431,3.10992,4.14056,4.88988,100.493,"[1, 1, 1]",compression,4.878898
0,3.1431,3.10992,4.14056,4.88988,100.493,"[0, 1, 1]",tension,10.008186
0,3.1431,3.10992,4.14056,4.88988,100.493,"[0, 1, 1]",compression,6.849351


Calculate actual $\lambda_1$, $\lambda_2$, and $\lambda_3$

In [6]:
data = {
    "a0 (A)": [3.1431],
    "a (A)": [3.10992],
    "b (A)": [4.14056],
    "c (A)": [4.88988],
    "beta": [100.493],
}
df = DataHandler(data).generate_dataframe()
LambdaCalculator(df).generate_lambdas()


,a0 (A),a (A),b (A),c (A),beta,lambda1_calculated,lambda2_calculated,lambda3_calculated
0,3.1431,3.10992,4.14056,4.88988,100.493,0.930119,0.931506,1.150677


Predict $\lambda_2$

In [7]:
data = {"Cu": [8], "Hf": [27], "Ni": [41.9], "Ti": [23.1]}

data_handler = DataHandler(data)
df = data_handler.generate_dataframe()

feature_generator = FeatureGenerator(df)

comp_df = feature_generator.generate_composition_formula()
features_df = feature_generator.generate_features()

lambda2_model = Lambda2Model(features_df)

prediction_df = lambda2_model.predict_transformation_and_lambda2()
prediction_df


,composition,Alloy_System,jarvis_avg_first_ion_en_divi_voro_coord,Predicted_Transformation_Type,Predicted_Lambda2
0,Cu8.0Hf27.0Ni41.9Ti23.1,NiTiHfCu,0.647085,B19',0.92


Machine learning model for estimating theoretical transformation strain

In [8]:
# Read data
df = pd.read_csv("data_for_ml.csv")
df["deformation_direction"] = df["deformation_direction"].apply(ast.literal_eval)

# Select columns
df = df[
    [
        "Predicted_Lambda2",
        "deformation_direction",
        "deformation_type",
        "LDT_transformation_strain",
    ]
].reset_index(drop=True)

# Split data into features and target
X = df[["Predicted_Lambda2", "deformation_direction", "deformation_type"]]
y = df["LDT_transformation_strain"]

# Initialize the CatBoostRegressor
model = CatBoostRegressor(
    silent=True,
    cat_features=["deformation_type"],
    embedding_features=["deformation_direction"],
)


Cross validation of the machine learning model

In [9]:
# Define a KFold cross-validator
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store the scores
scores = {
    "r2_test": [],
    "r2_train": [],
    "mae_test": [],
    "mae_train": [],
    "rmse_test": [],
    "rmse_train": [],
}

# Perform KFold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    # Calculate and store the scores for both training and validation data
    scores["r2_train"].append(r2_score(y_train, y_pred_train))
    scores["r2_test"].append(r2_score(y_test, y_pred_test))
    scores["mae_train"].append(mean_absolute_error(y_train, y_pred_train))
    scores["mae_test"].append(mean_absolute_error(y_test, y_pred_test))
    scores["rmse_train"].append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    scores["rmse_test"].append(np.sqrt(mean_squared_error(y_test, y_pred_test)))

# Print the results with standard deviations
for metric, values in scores.items():
    print(f"{metric.replace('_', ' ').title()} Scores: {values}")
    print(
        f"Average {metric.replace('_', ' ').title()}: {np.mean(values):.3f}, "
        f"Std Dev: {np.std(values):.3f}\n"
    )


R2 Test Scores: [0.9646551474490295, 0.9595241211669925, 0.9609411965227928, 0.9470787913592512, 0.9655762221061949, 0.9475184743734233, 0.9459237866127591, 0.940790670445635, 0.9474007229890613, 0.9611857303604322]
Average R2 Test: 0.954, Std Dev: 0.009

R2 Train Scores: [0.960705137174717, 0.9612447647691571, 0.9605475312643229, 0.9625011310448013, 0.9605433212277124, 0.9625669318092389, 0.9623539885692862, 0.9630193623538914, 0.9621275107114324, 0.9610609617023244]
Average R2 Train: 0.962, Std Dev: 0.001

Mae Test Scores: [0.5822740416269253, 0.5929110761797515, 0.578850893477759, 0.6334286955642991, 0.5534035192917592, 0.6658036669797027, 0.5906181618497359, 0.6877324804805393, 0.632958837357053, 0.5656775551567512]
Average Mae Test: 0.608, Std Dev: 0.042

Mae Train Scores: [0.5472957410130723, 0.5480059995406048, 0.5594598663717407, 0.5405245228848571, 0.5538583519531307, 0.5414515806321171, 0.551839472769351, 0.538670486000266, 0.5490783031829023, 0.5542659833836939]
Average Mae 